# 02 - Kat subset

In [3]:
import anndata
from anndata import AnnData
import scanpy.api as sc
import pandas as pd
import numpy as np
import inspect
import sys
import copy
import datetime
import matplotlib.pylab as plt
import natsort
from IPython.display import clear_output
counter = lambda x: pd.DataFrame(zip(*np.unique([str(i) for i in x], return_counts = True)), columns = ['name', 'counts'])

/opt/conda/envs/scanpy-1.4.4/lib/python3.7/site-packages/scanpy/api/__init__.py:6: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning


In [2]:
def preprocess(Loading_file = None,
               adata = None,
               minimum_percent_of_cell_should_express_gene = 0.0500001,
               Vivo_vitro_silico = None,
               normalize = 1000,
               verbose = False,
               minumum_number_of_genes_expressed = None,
               filter_cells = False,
               log = False,
               clustering = True,
               upper_bound = 2,
               try_to_exclude_cells_not_used_in_analysis = True,
               **kwargs):
    """Returns a dictionary containing 'anndata' and 'Preprocess parameters'.
    
       Loading_file = 
         '../3_Loading/6-Scialdone2016-anndata.h5ad' # Example
                      
       minimum_percent_of_cell_should_express_gene = 0.050001
         is used to clean out unexpressed genes. At least 5% cells must express.
       
       normalize = 1000
         normalizes so each cell have a total expression of 1000"""

   
    args, _, _, values = inspect.getargvalues(inspect.currentframe()); 
    input_params = {**{arg: values[arg] for arg in args  if sys.getsizeof(values[arg]) < 100000}, 
                    **{'EXTRA_'+arg: values['kwargs'][arg] for arg in kwargs.keys() if sys.getsizeof(values['kwargs'][arg]) < 100000}}


    metric_holder = {}
    
    
    if verbose > 0: start = datetime.datetime.now()
    if verbose > 0: print('Preprocessing started..: ', start.strftime("%H:%M"), '~' ,(start + datetime.timedelta(minutes = 0.10)).strftime("%H:%M"), end=' ')
    
    
    if verbose > 0:
        def timer():
            curtime = datetime.datetime.now().strftime("%H:%M")
            diff = (datetime.datetime.now() - start)
            difftime = ':'.join(str(diff).split(':')[:2])
            return(' # Time:' + curtime + '. Elapsed: ' + difftime)
    

    ### Loading
    if Loading_file != None:
        adata = sc.read_h5ad(Loading_file)
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        if verbose >= 3:
            print('Loaded data' + timer())
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))
            
            
    if adata != None:
        adata = adata
        if verbose >= 3:
            print('Loaded data' + timer())
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))

    adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
    adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
       
    if try_to_exclude_cells_not_used_in_analysis:
        to_exclude = ['mars2' in i.lower() or 'doublet' in i.lower() for i in adata.obs['Condition_Condition']] # '2ilif' in i.lower() or
        adata = AnnData(adata[~np.array(to_exclude)])
        
        
    metric_holder['Raw'] = return_metric_after(adata)
        
        
    ### Remove not expressed genes
    if minimum_percent_of_cell_should_express_gene != None and minimum_percent_of_cell_should_express_gene != 0:
        n_cells = len(adata.obs)
        if verbose >= 3:
            print('### Remove not expressed genes' + timer())
        sc.pp.filter_genes(adata, min_cells=n_cells * minimum_percent_of_cell_should_express_gene)
        if verbose >= 5:
            print('fixing indices')
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        if verbose >= 3:
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))
        metric_holder['Removed non-expressing genes'] = return_metric_after(adata)

        
    ### Take only "In vivo" ### DO NOT USE. For legacy purpose.
    # This creates a view of adata, not sure if that have any consequences...
    if type(Vivo_vitro_silico) == list:
        if verbose >= 3:
            print('### Only In vivo' + timer())
        adata = adata[ adata.obs['Vivo_vitro_silico'].isin(Vivo_vitro_silico) , : ]
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        if verbose >= 3:
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))
        
        
    ### Remove non-expressing cells
    if minumum_number_of_genes_expressed != None:
        if verbose >= 3:
            print('### Removing non-expressing genes' + timer())
        sc.pp.filter_cells(adata, min_genes = minumum_number_of_genes_expressed)
        metric_holder['Removed non-expressing cells'] = return_metric_after(adata)        
        
        
    sc.pp.calculate_qc_metrics(adata, inplace=True)    
    if filter_cells == True:
        if verbose >= 3:
            print('### Filtering cells' + timer())
        if verbose >= 5:
            print('qc metrics' + timer())
            
        
        if verbose >= 5:
            print('Medians of counts and n-genes' + timer())        
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))
            
            
        total_median = np.median(adata.obs['total_counts'])
        total_std = np.std(adata.obs['total_counts'])
        total_max = np.max(adata.obs['total_counts'])
        total_median_minus_std = total_median - total_std
        total_median_plus_std  = total_median + upper_bound * total_std

        
        n_median = np.median(adata.obs['n_genes_by_counts'])
        n_std = np.std(adata.obs['n_genes_by_counts'])
        n_max = np.max(adata.obs['n_genes_by_counts'])
        n_median_minus_std = n_median-n_std
        n_median_plus_std  = n_median + upper_bound * n_std

        
        if verbose >= 1:
            print('n_median:', n_median)
            print('n_std:', n_std)
            print('n_max:', n_max)
            print('n_median_minus_std:', n_median_minus_std)
            print('n_median_plus_std:', n_median_plus_std)
        
            print('total_median:', total_median)
            print('total_std:', total_std)
            print('total_max:', total_max)
            print('total_median_minus_std:', total_median_minus_std)
            print('total_median_plus_std:', total_median_plus_std)
        
        
        metric_holder['Before filtering'] = return_metric_after(adata)
        metric_holder['Cutoffs'] = {'total_median' : total_median, 
                                    'total_std' : total_std, 
                                    'total_max' : total_max, 
                                    'total_median_minus_std' : total_median_minus_std, 
                                    'total_median_plus_std' : total_median_plus_std, 
                                    'n_median' : n_median, 
                                    'n_std' : n_std, 
                                    'n_max' : n_max, 
                                    'n_median_minus_std' : n_median_minus_std, 
                                    'n_median_plus_std' : n_median_plus_std,
                                   }
        
        
        if verbose >= 5:
            print('Removing only top 500 genes expressing cells' + timer())
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))
        adata = AnnData(adata[adata.obs['pct_counts_in_top_500_genes'] < (np.median(adata.obs['pct_counts_in_top_500_genes']) + 0.5 *np.std(adata.obs['pct_counts_in_top_500_genes']))])
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        metric_holder['Removed only top 500 genes expressing cells'] = return_metric_after(adata)
        
        
        if verbose >= 5:
            print('Removing cells with too few n_genes' + timer())
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))
        sc.pp.filter_cells(adata, min_genes = n_median_minus_std)
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        metric_holder['Removed cells with too few n_genes'] = return_metric_after(adata)
        metric_holder['Removed cells with too many total counts _ min_genes'] = n_median_minus_std

        
        if upper_bound != -1:
            if verbose >= 5:
                print('Removing genes with too many n_genes' + timer())
                print('genes:' ,len(adata.var))
                print('cells:' ,len(adata.obs))
            sc.pp.filter_cells(adata, max_genes = n_median_plus_std)
            adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
            adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
            metric_holder['Removed genes with too many n_genes'] = return_metric_after(adata)
            metric_holder['Removed cells with too many total counts _ max_genes'] = n_median_plus_std
        
        
        if verbose >= 5:
            print('Removing genes with too few counts' + timer())
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))
        sc.pp.filter_cells(adata, min_counts = total_median_minus_std)
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        metric_holder['Removed cells with too few total counts'] = return_metric_after(adata)
        metric_holder['Removed cells with too many total counts _ min_counts'] = total_median_minus_std
        
        
        if upper_bound != -1:
            if verbose >= 5:
                print('Removing genes with too many counts' + timer())
                print('genes:' ,len(adata.var))
                print('cells:' ,len(adata.obs))
            sc.pp.filter_cells(adata, max_counts = total_median_plus_std)        
            adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
            adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
            metric_holder['Removed cells with too many total counts'] = return_metric_after(adata)
            metric_holder['Removed cells with too many total counts _ max_counts'] = total_median_plus_std
        
        if verbose >= 5:
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))        
        
    ### Normalize per cell
    if normalize != None and normalize != False:
        if verbose >= 3:
            print('Normalizing cells' + timer())
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        adata = sc.pp.normalize_per_cell(adata, counts_per_cell_after=normalize, copy=True)
        adata.var.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.var.index), step=1)]
        adata.obs.index = [str(i) for i in pd.RangeIndex(start=0, stop=len(adata.obs.index), step=1)]
        if verbose >= 4:
            print('genes:' ,len(adata.var))
            print('cells:' ,len(adata.obs))

    if log: 
        if verbose >= 3:
            print('### Taking log1p' + timer())
        sc.pp.log1p(adata)
        
            
    if clustering:
        if verbose >= 3:
            print('### Doing a lot of clustering' + timer())
        if verbose >= 5:
            print('Find neighbors' + timer())
        sc.pp.neighbors(adata, method='umap')
        if verbose >= 5:
            print('1/6 - louvain 0.65...' + timer())
        sc.tl.louvain(adata, key_added = 'Clustering_louvain_0.65', resolution = 0.65)
        if verbose >= 5:
            print('2/6 - leiden 0.65...' + timer())
        sc.tl.leiden(adata, key_added = 'Clustering_leiden_0.65', resolution = 0.65)
        if verbose >= 5:
            print('3/6 - louvain 0.75...' + timer())
        sc.tl.louvain(adata, key_added = 'Clustering_louvain_0.75', resolution = 0.75)
        if verbose >= 5:
            print('4/6 - leiden 0.75...' + timer())
        sc.tl.leiden(adata, key_added = 'Clustering_leiden_0.75', resolution = 0.75)
        if verbose >= 5:
            print('5/6 - louvain 0.85...' + timer())
        sc.tl.louvain(adata, key_added = 'Clustering_louvain_0.85', resolution = 0.85)
        if verbose >= 5:
            print('6/6 - leiden 0.85...' + timer())
        sc.tl.leiden(adata, key_added = 'Clustering_leiden_0.85', resolution = 0.85)
        if verbose >= 5:
            print('6/6 - DONE!' + timer())
        
        
    if verbose > 0: print("(%d min, %d sec)" % ((datetime.datetime.now()-start).seconds%3600/60, (datetime.datetime.now()-start).seconds%60))
    
    
    metric_holder['Preprocess end'] = return_metric_after(adata)

        
    return {'adata': adata, 'Preprocess_parameters': input_params, 'metric': metric_holder}

In [7]:
def read(filter_cells = True, minimum_percent_of_cell_should_express_gene = 0.0500001):
    """Read"""
    "Takes about 10 sec"
    Kat = sc.read_h5ad('../data/external/nowotschin_et_al_2019/sc_endoderm_all_cells.h5ad') 
    Kat.var['ID_Symbol'] = Kat.var.index.values
    Kat.obs['barcode'] = np.array(Kat.obs.index)
    prep = preprocess.preprocess(adata = AnnData(Kat), clustering=False, normalize=1, filter_cells = filter_cells, 
                                 minimum_percent_of_cell_should_express_gene = minimum_percent_of_cell_should_express_gene, 
                                 try_to_exclude_cells_not_used_in_analysis=False)
    Katp = prep['adata']
    display(Kat)
    display(Katp) # Kat processed


    """Make sure the normalization is good"""
    assert(np.all(np.abs(Katp.X.sum(axis = 1) - 1 ) < 0.001))
    #display(Katp.var)

    
    """Proper var index"""
    Kat.var.index = Kat.var['ID_Symbol']

    
    """Timepoints"""
    set(Katp.obs['Timepoint']);
    
    
    return Kat, Katp

In [ ]:
### THIS STEPS NEEDS TO BE DONE MANUALLY
"""input_genes = Kat.var['ID_Symbol']""" # Use these to generate the MGIBatchReport
# with open('./Kat genes.txt', 'w') as f:
#     for item in Katp.var['ID_Symbol']:
#         f.write("%s\n" % item)


"""Went to http://www.informatics.jax.org/batch and converted the genes above"""
Genes_DIFFERENT_TYPE_IDS_df = pd.read_table('../data/external/MGIBatchReport_20191015_073607.txt')
#display(Genes_DIFFERENT_TYPE_IDS_df.head())


In [8]:
"""Define function to convert the genenames based on MGIs conversion batch report"""


def convert_officialgeneid(name):
    
    """Define preferred gene match from MGI"""
    ordered_preference_list = ['current symbol',
                               'old symbol',
                               'synonym',
                               'human symbol',
                               'human synonym',
                               'related synonym',
                               'rat synonym',
                               'GenBank',
                               'zebrafish symbol',
                               'rat symbol',
                               'cattle symbol',
                               'macaque, rhesus symbol',
                               'chicken symbol',
                               'chimpanzee symbol',
                               'dog symbol',
                               'frog, western clawed symbol']


    """If there is some input type that we have not considered, then add it to the buttom of the list of preferences"""
    for i in list(set(Genes_DIFFERENT_TYPE_IDS_df['Input Type'])):
        if i not in ordered_preference_list:
            ordered_preference_list.append(i)
    
    df = Genes_DIFFERENT_TYPE_IDS_df 
    input_types = list(df[df['Input'] == name]['Input Type']) # 


    for i in range(len(ordered_preference_list)):    
        if ordered_preference_list[i] in input_types:
            choose = ordered_preference_list[i]
            try:
                return (df.loc[(df['Input'] == name) & (df['Input Type'] == choose)].iloc[0])
            except:
                pass



    dummy_df = pd.DataFrame(data = [[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]], 
                            columns=['Input', 'Input Type', 'MGI Gene/Marker ID', 'Symbol', 'Name', 'Feature Type', 'Ensembl ID', 'Entrez Gene ID', 'VEGA ID'])


    dummy_df.loc[0, 'Input'] = name
    dummy_df.loc[0, 'Input Type'] = 'not found symbol'


    return dummy_df.iloc[0,:]


def convert_gene_names(input_genes):
    '''Potentially takes a few minutes to run'''

    """Run"""
    gene = input_genes[0]
    ID = convert_officialgeneid(gene)
    df_ID = pd.DataFrame(ID).transpose()


    import multiprocessing
    pool = multiprocessing.Pool()
    res = pool.map(convert_officialgeneid, input_genes)
    var = pd.concat([pd.DataFrame(elem).transpose() for elem in res])
    pool.close()


    """Rename the var columns"""
    Rename_var_keys = {'Input'              : 'Info_Raw Data Input', 
                       'Input Type'         : 'Info_Raw Data Input Type',
                       'MGI Gene/Marker ID' : 'ID_MGI Gene/Marker',
                       'Symbol'             : 'ID_Symbol',
                       'Name'               : 'Info_Full Name',
                       'Feature Type'       : 'Info_Feature Type',
                       'Ensembl ID'         : 'ID_Ensembl ID',
                       'Entrez Gene ID'     : 'ID_Entrez Gene ID',
                       'VEGA ID'            : 'ID_VEGA ID'}


    try: var.columns = [Rename_var_keys[key] for key in var.columns]
    except Exception as e: print(e)
    var.head(5)
    print(len(var))


    """Inspect the new var"""
    display(var.head(5))


    """Show genes that did not get a proper conversion"""
    a, b = np.unique([i for i in var['ID_Symbol']], return_counts=True)
    for gene in a[b > 1]:
        display(var[var['ID_Symbol'].astype(str) == gene])


    """Make sure the new 'var' df comes in the same order as the input data"""
    assert(np.all(np.array(input_genes) == np.array(var['Info_Raw Data Input'])))


    """return"""
    return var

In [ ]:
def subsample_with_minimum(Katp, minimum_desired_samples = 100, n_subsample = 10000):
    indices = []
    indices_not_used = []
    for cluster in set(Katp.obs['CellType']):
        Katp_cluster = Katp[Katp.obs['CellType'] == str(cluster)]
        if Katp_cluster.shape[0] > 0:
            sub = sc.pp.subsample(Katp_cluster, n_obs=np.min([Katp_cluster.shape[0], minimum_desired_samples]), random_state=42, copy=True)
            indices += list(sub.obs.index)
            indices_not_used += list(set(Katp_cluster.obs.index) - set(sub.obs.index))

    np.random.seed(42)
    indices_additional = list(np.random.choice(indices_not_used, n_subsample-len(indices), replace = False))
    Katp_subsampled = AnnData(Katp[indices+indices_additional])
    return Katp_subsampled


def tsne_and_subsample(adata, n_subsample = 10000, min_sample = 200):    
    """Subsample the data and run tsne"""
    #adata_sub = sc.pp.subsample(adata, n_obs=n_subsample, copy = True)
    #sc.tl.tsne(adata_sub)
    adata_sub = subsample_with_minimum(adata, n_subsample=n_subsample, minimum_desired_samples=min_sample)
    
    X_embedded = TSNE(n_components=2, perplexity=15, learning_rate=10, random_seed = 42).fit_transform(adata_sub.X.toarray())
    adata_sub.obsm['X_tsne'] =  X_embedded
    

    """Make all the different 'Gut tube' types into one type"""
    def CellTyper(i):
        if "gut" in str(i).lower():
            return "Gut"
        else:
            return str(i)
    adata_sub.obs["CellType simple"] = [CellTyper(i) for i in adata_sub.obs["CellType"]]
    
    return adata_sub

In [ ]:
def select_foxa2_clusters(Katp):
        #Foxa2_df[Foxa2_df['Fraction'] < 0.2].index.tolist()
        Not_Foxa2 = ['Blood',
                     'Gut tube:DE:Thymus',
                     'TE',
                     'Gut tube:DE:Thyroid',
                     'Gut tube:VE:Thyroid',
                     'GermCells',
                     'ExE',
                     'Mes',
                     'EPI',
                     'Endothelial',
                     'nan']


        #Foxa2_df[Foxa2_df['Fraction'] > 0.2].index.tolist()
        Yes_Foxa2 = ['PrE',
                     'exVE',
                     'Gut tube:VE:Lung',
                     'Gut tube:VE:Small int',
                     'Gut tube:DE:Lung',
                     'emVE',
                     'Midline',
                     'DE',
                     'Gut tube:VE:Liver',
                     'Gut tube:DE:Colon',
                     'Gut tube:VE:Colon',
                     'Gut tube:DE:Pancreas',
                     'Gut tube:DE:Small int',
                     'VE',
                     'Gut tube',
                     'Gut tube:VE:Pancreas',
                     'ParE',
                     'YsE',
                     'Gut tube:DE:Liver',
                     'Gut tube:VE:Thymus']


        Katp_Foxa2plus = Katp[np.isin(Katp.obs['CellType'], Yes_Foxa2)]
        print('Before', Katp.n_obs)
        print('Foxa2+ ', Katp_Foxa2plus.n_obs, '<-- Using these')
        print('Foxa2- ', Katp.n_obs - Katp_Foxa2plus.n_obs)
        
        
        return Katp_Foxa2plus

In [ ]:
if __name__ == '__main__':

    n_subsample = 10000
    foxa2plus_only = False
    save_filename = 'Kat_subset_foxa2plus_more_genes_less_filter_even_more_clusters_mid03_mes03_emVE02.h5ad'


    Kat, Katp = read(filter_cells=False, minimum_percent_of_cell_should_express_gene = 0.0010001)


    """Fix nan -> ICM"""
    CellType = np.array(Katp.obs['CellType'])
    CellType[[str(i) == 'nan' for i in Katp.obs['CellType']]] = 'ICM'
    Katp.obs['CellType'] = CellType


    """convert genenames"""
    var = convert_gene_names(Katp.var['ID_Symbol'])
    var.index = var['Info_Raw Data Input'].astype(str)
    Katp.var = var


    """Subclustering of EmVE and Mes"""
    res = 0.2
    #res = 0.03


    Katp_emVE     = AnnData(Katp[[i in ['emVE'] for i in Katp.obs['CellType']]])
    sc.pp.neighbors(Katp_emVE)
    sc.tl.leiden(Katp_emVE, resolution=res) # 7 - 0.2

    Katp_mes      = AnnData(Katp[[i in ['Mes'] for i in Katp.obs['CellType']]])
    sc.pp.neighbors(Katp_mes)
    sc.tl.leiden(Katp_mes, resolution=0.3) # 7 - 0.3

    Katp_Midline     = AnnData(Katp[[i in ['Midline'] for i in Katp.obs['CellType']]])
    sc.pp.neighbors(Katp_Midline)
    sc.tl.leiden(Katp_Midline, resolution=0.3) # 7 - 0.3

    Katp_VELung     = AnnData(Katp[[i in ['Gut tube:VE:Lung'] for i in Katp.obs['CellType']]])
    sc.pp.neighbors(Katp_VELung)
    sc.tl.leiden(Katp_VELung, resolution=res)

    Katp_DELung     = AnnData(Katp[[i in ['Gut tube:DE:Lung'] for i in Katp.obs['CellType']]])
    sc.pp.neighbors(Katp_DELung)
    sc.tl.leiden(Katp_DELung, resolution=res)

    clear_output()

    Katp.obs['CellType_before_subclustering'] = np.array(Katp.obs['CellType'])
    Katp.obs['CellType'] = np.array(Katp.obs['CellType']) # To change the categorical type to object
    Katp.obs.loc[Katp_emVE.obs.index, ['CellType']] = ['emVE'+i for i in Katp_emVE.obs['leiden']]
    Katp.obs.loc[Katp_mes.obs.index, ['CellType']] = ['mes'+i for i in Katp_mes.obs['leiden']]
    Katp.obs.loc[Katp_Midline.obs.index, ['CellType']] = ['Midline'+i for i in Katp_Midline.obs['leiden']]
    #Katp.obs.loc[Katp_VELung.obs.index, ['CellType']] = ['VELung'+i for i in Katp_VELung.obs['leiden']]
    #Katp.obs.loc[Katp_DELung.obs.index, ['CellType']] = ['DELung'+i for i in Katp_DELung.obs['leiden']]


    # """Select Foxa2+ clusters"""
    # if foxa2plus_only:
    #     Katp_plus = select_foxa2_clusters(Katp)


    """Subsample and TSNE"""
    adata_sub = tsne_and_subsample(adata = Katp, n_subsample = n_subsample, min_sample = 200)
    Katp_sub = AnnData(adata_sub)
    display(Katp)
    display(Katp_sub)


    """Write out"""
    #adata2 = write_out(adata = Katp_sub, filename = save_filename)